# Aplicando as técnicas no conjunto de dados do MNIST

## Importando o Dataset e fazendo manipulações

In [1]:
import keras.datasets
from keras.datasets import mnist     # Dataset MNIST direto do Keras
import numpy as np
import random
import seaborn
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from sklearn.utils import resample
from sklearn.base import BaseEstimator, ClassifierMixin, RegressorMixin
from sklearn.model_selection import train_test_split
from keras.utils import np_utils                         # Ferramentas relacionadas ao NumPy

In [2]:
# Os dados do MNIST jávem separados em 60,000 imagens de treino (28x28 pixels) e 10,000 imagens para teste (28x28 pixels).

(X_train, y_train), (X_test, y_test) = mnist.load_data() # Separando os dados em treino e teste

print("X_train shape", X_train.shape)
print("y_train shape", y_train.shape)
print("X_test shape", X_test.shape)
print("y_test shape", y_test.shape)

X_train shape (60000, 28, 28)
y_train shape (60000,)
X_test shape (10000, 28, 28)
y_test shape (10000,)


In [3]:
X_train = X_train.reshape(60000, 784) # remodelar(reshape) 60.000 matrizes 28 x 28 em 60.000 vetores de comprimento de 784.
X_test = X_test.reshape(10000, 784)   # remodelar(reshape) 10.000 matrizes 28 x 28 em 10.000 vetores de comprimento de 784.

X_train = X_train.astype('float32')   # alterando para ponto flutuante (float) de 32 bits
X_test = X_test.astype('float32')

X_train /= 255          # máximo valor de um pixel (normalização)
X_test /= 255

print("Training matrix shape", X_train.shape)
print("Testing matrix shape", X_test.shape)

Training matrix shape (60000, 784)
Testing matrix shape (10000, 784)


In [4]:
n_classes = 10 # número de dígitos únicos

Y_train = np_utils.to_categorical(y_train, n_classes)
Y_test = np_utils.to_categorical(y_test, n_classes)

## Perceptron Multicamada:

In [5]:
class MultiLayerPerceptron(BaseEstimator, ClassifierMixin): 
    def __init__(self, params=None):     
        if (params == None):
            self.inputLayer = 30                        # Camada de Input
            self.hiddenLayer = 1                       # Camadas Ocultas
            self.OutputLayer = 10                       # Camadas de Output
            self.learningRate = 0.005                  # Taxa de aprendizado
            self.max_epochs = 600                      # Épocas
            self.BiasHiddenValue = -1                   # Bias das Camadas Ocultas
            self.BiasOutputValue = -1                  # Bias da Camada de Output
            self.activation = self.ativacao['sigmoid'] # Função de Ativação
            self.deriv = self.derivada['sigmoid']
        else:
            self.inputLayer = params['InputLayer']
            self.hiddenLayer = params['HiddenLayer']
            self.OutputLayer = params['OutputLayer']
            self.learningRate = params['LearningRate']
            self.max_epochs = params['Epocas']
            self.BiasHiddenValue = params['BiasHiddenValue']
            self.BiasOutputValue = params['BiasOutputValue']
            self.activation = self.ativacao[params['ActivationFunction']]
            self.deriv = self.derivada[params['ActivationFunction']]
        
        'Iniciando pesos e bias aleatoriamente'
        self.WEIGHT_hidden = self.starting_weights(self.hiddenLayer, self.inputLayer)
        self.WEIGHT_output = self.starting_weights(self.OutputLayer, self.hiddenLayer)
        self.BIAS_hidden = np.array([self.BiasHiddenValue for i in range(self.hiddenLayer)])
        self.BIAS_output = np.array([self.BiasOutputValue for i in range(self.OutputLayer)])
        self.classes_number = 10 
        
    pass
    
    def starting_weights(self, x, y):
        return [[2  * random.random() - 1 for i in range(x)] for j in range(y)]

    ativacao = {
         'sigmoid': (lambda x: 1/(1 + np.exp(-x))),
            'tanh': (lambda x: np.tanh(x)),
            'Relu': (lambda x: x*(x > 0)),
               }
    derivada = {
         'sigmoid': (lambda x: x*(1-x)),
            'tanh': (lambda x: 1-x**2),
            'Relu': (lambda x: 1 * (x>0))
               }
 
    def Backpropagation_Algorithm(self, x):
        DELTA_output = []
        'Fase 1 - Erro: OutputLayer'
        ERROR_output = self.output - self.OUTPUT_L2
        DELTA_output = ((-1)*(ERROR_output) * self.deriv(self.OUTPUT_L2))
        
        arrayStore = []
        'Fase 2 - Atualizando pesos da OutputLayer e HiddenLayer'
        for i in range(self.hiddenLayer):
            for j in range(self.OutputLayer):
                self.WEIGHT_output[i][j] -= (self.learningRate * (DELTA_output[j] * self.OUTPUT_L1[i]))
                self.BIAS_output[j] -= (self.learningRate * DELTA_output[j])
      
        'Fase 3 - Erro: HiddenLayer'
        delta_hidden = np.matmul(self.WEIGHT_output, DELTA_output)* self.deriv(self.OUTPUT_L1)
 
        'Fase 4 - Atualizando pesos da HiddenLayer e InputLayer(x)'
        for i in range(self.OutputLayer):
            for j in range(self.hiddenLayer):
                self.WEIGHT_hidden[i][j] -= (self.learningRate * (delta_hidden[j] * x[i]))
                self.BIAS_hidden[j] -= (self.learningRate * delta_hidden[j])
                
 #   def show_err_graphic(self,v_erro,v_epoca):
 #       plt.figure(figsize=(9,4))
 #       plt.plot(v_epoca, v_erro, "m-",color="b", marker=11)
 #       plt.xlabel("Number of Epochs")
 #       plt.ylabel("Squared error (MSE) ");
 #       plt.title("Error Minimization")
 #       plt.show()

    def predict(self, X, y):
        'Retorna a predição pra cada valor de X'
        my_predictions = []
        'Algoritmo Forward'
        forward = np.matmul(X,self.WEIGHT_hidden) + self.BIAS_hidden
        forward = np.matmul(forward, self.WEIGHT_output) + self.BIAS_output
                                 
        for i in forward:
            my_predictions.append(max(enumerate(i), key=lambda x:x[1])[0])
            
        array_score = []
        for i in range(len(my_predictions)):
            if my_predictions[i] == 0: 
                array_score.append([i, 'Zero', my_predictions[i], y[i]])
            elif my_predictions[i] == 1:
                 array_score.append([i, 'Um', my_predictions[i], y[i]])
            if my_predictions[i] == 2: 
                array_score.append([i, 'Dois', my_predictions[i], y[i]])
            elif my_predictions[i] == 3:
                 array_score.append([i, 'Três', my_predictions[i], y[i]])
            if my_predictions[i] == 4: 
                array_score.append([i, 'Quatro', my_predictions[i], y[i]])
            elif my_predictions[i] == 5:
                 array_score.append([i, 'Cinco', my_predictions[i], y[i]])
            if my_predictions[i] == 6: 
                array_score.append([i, 'Seis', my_predictions[i], y[i]])
            elif my_predictions[i] == 7:
                 array_score.append([i, 'Sete', my_predictions[i], y[i]])
            if my_predictions[i] == 8: 
                array_score.append([i, 'Oito', my_predictions[i], y[i]])
            elif my_predictions[i] == 9:
                 array_score.append([i, 'Nove', my_predictions[i], y[i]])
                    
        dataframe = pd.DataFrame(array_score, columns=['_id', 'class', 'output', 'hoped_output'])
        return my_predictions, dataframe

    def fit(self, X, y):  
        count_epoch = 1
        total_error = 0
        n = len(X); 
        epoch_array = []
        error_array = []
        W0 = []
        W1 = []
        while(count_epoch <= self.max_epochs):
            for idx,inputs in enumerate(X): 
                self.output = np.zeros(self.classes_number)
                'Fase 1 - (Algoritmo Forward)'
                self.OUTPUT_L1 = self.activation((np.dot(inputs, self.WEIGHT_hidden) + self.BIAS_hidden.T))
                self.OUTPUT_L2 = self.activation((np.dot(self.OUTPUT_L1, self.WEIGHT_output) + self.BIAS_output.T))
                'Fase 2 - One-Hot-Encoding'
                if(y.any() == 0): 
                    self.output = np.array([1,0,0,0,0,0,0,0,0,0])
                elif(y.any() == 1):
                    self.output = np.array([0,1,0,0,0,0,0,0,0,0]) 
                if(y.any() == 2): 
                    self.output = np.array([0,0,1,0,0,0,0,0,0,0])
                elif(y.any() == 3):
                    self.output = np.array([0,0,0,1,0,0,0,0,0,0]) 
                if(y.any() == 4): 
                    self.output = np.array([0,0,0,0,1,0,0,0,0,0])
                elif(y.any() == 5):
                    self.output = np.array([0,0,0,0,0,1,0,0,0,0]) 
                if(y.any() == 6): 
                    self.output = np.array([0,0,0,0,0,0,1,0,0,0])
                elif(y.any() == 7):
                    self.output = np.array([0,0,0,0,0,0,0,1,0,0]) 
                if(y.any() == 8): 
                    self.output = np.array([0,0,0,0,0,0,0,0,1,0])
                elif(y.any() == 9):
                    self.output = np.array([0,0,0,0,0,0,0,0,0,1]) 

                
                square_error = 0
                for i in range(self.OutputLayer):
                    erro = (self.output[i] - self.OUTPUT_L2[i])**2
                    square_error = (square_error + (0.05 * erro))
                    total_error = total_error + square_error
         
                'Backpropagation : Atualização de pesos'
                self.Backpropagation_Algorithm(inputs)
                
            total_error = (total_error / n)
            if((count_epoch % 10 == 0)or(count_epoch == 1)):
                print("Época ", count_epoch, "- Erro Total: ",total_error, f'\n Acurácia: {round((1-total_error)*100,2)}%')
                error_array.append(total_error)
                epoch_array.append(count_epoch)
                
            W0.append(self.WEIGHT_hidden)
            W1.append(self.WEIGHT_output)
             
                
            count_epoch += 1
#        self.show_err_graphic(error_array,epoch_array)
        
#        plt.plot(W0[0])
#        plt.title('Weight Hidden update during training')
#        plt.legend(['neuron1', 'neuron2', 'neuron3', 'neuron4', 'neuron5'])
#        plt.ylabel('Value Weight')
#        plt.show()
        
#        plt.plot(W1[0])
#        plt.title('Weight Output update during training')
#        plt.legend(['neuron1', 'neuron2', 'neuron3'])
#        plt.ylabel('Value Weight')
#        plt.show()

        return self

## Testando

In [21]:
def acuracia(y_true, y_pred):
    acuracia = np.sum(y_true == y_pred) / len(y_true)
    return acuracia

In [26]:
dic = {'InputLayer':784, 'HiddenLayer':1, 'OutputLayer':10,
              'Epocas':10, 'LearningRate':0.01,'BiasHiddenValue':-1, 
              'BiasOutputValue':-1, 'ActivationFunction':'tanh'}

Perceptron = MultiLayerPerceptron(dic)
Perceptron.fit(X_train,Y_train)

#import warnings
#warnings.filterwarnings('ignore')

Época  1 - Erro Total:  0.045242697606371195 
 Acurácia: 95.48%


KeyboardInterrupt: 

Mesmo com somente 10 épocas, o tempo de execução foi extremamente alto, o que acabou começando a comprometer o meu computador, então a execução foi parada. Não se sabe quantas épocas foram feitas, sabe-se que foi algo entre 1 e 9 épocas